There are two tasks that I need to complete for this week.

Firstly is building my own learning class/object/thingy.

Secondly I need to use this new learner object to create a model to classify the MNIST dataset.

Exciting!


In [9]:
# Needed modules

import torch
import numpy as np
from fastai.vision.all import *

from torch import nn
import matplotlib.pyplot as plt

# built int

import os

In [172]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## Building learner

A learner needs 4 bits of information to work
- Data
- Model
- Optimizer
- Loss function

### Getting data

In [43]:
class MyOptimizer:
    def __init__(self, params, lr):
        self.params = list(params)
        self.lr = lr

    def step(self):
        for p in self.params:
            with torch.no_grad():
                p -= self.lr * p.grad

    def zero_grad(self):
        for p in self.params:
            if p.grad is not None:
                p.grad.zero_()

class MyLearner:
    def __init__(self, dls, model, loss_func, metric_func):
        self.dls = dls
        self.model = model
        self.loss_func = loss_func
        self.metric_func = metric_func

    def train_epoch(self, opt):
        updates = 0
        for xb, yb in self.dls.train:
            updates += 1
            pred = self.model(xb)
            loss = self.loss_func(pred, yb)
            loss.backward()
            opt.step()
            opt.zero_grad()
        print(f"Train loops: {updates}")
        return loss
    

    def validate_epoch(self):
        with torch.no_grad():
            accs = [self.metric_func(self.model(xb), yb) for xb,yb in self.dls.valid]
        return round(torch.stack(accs).mean().item(), 4) 

    
    def fit(self, epochs, lr):
        opt = MyOptimizer(self.model.parameters(), lr)
        opt.zero_grad()
        print("Epoch Train_Loss Valid_Accuracy  Time")
        for epoch in range(epochs):
            start = time.time()
            train_loss = self.train_epoch(opt)
            end = time.time()
            accuracy = self.validate_epoch()
            print(f" {epoch:02}:    {train_loss:.4f}     {accuracy:.4f}   {end-start:.4f}")

        
    def transform(self, x):
        return self.model(x)

    def predict(self, x):
        # Takes a tensor of shape (1,784) return a list of ints
        preds = self.transform(x).argmax(dim=0)
        return preds.tolist()


# 3 and 7 MNIST calssifier


## Getting data

In [3]:
path = untar_data(URLs.MNIST_SAMPLE)

### Training

In [4]:

threes = (path/'train'/'3').ls().sorted()
sevens = (path/'train'/'7').ls().sorted()

three_tensors = [tensor(Image.open(o)) for o in threes]
seven_tensors = [tensor(Image.open(o)) for o in sevens]

stacked_3d = torch.stack(three_tensors).float()/255
stacked_7d = torch.stack(seven_tensors).float()/255

train_x = torch.cat([stacked_3d, stacked_7d]).view(-1, 28*28)
train_y = tensor([1]*len(threes) + [0]*len(sevens)).unsqueeze(1)

dset = list(zip(train_x, train_y))
x,y = dset[0]
x.shape, y

(torch.Size([784]), tensor([1]))

In [5]:
dl = DataLoader(dset)

xb,yb = first(dl)
xb.shape, yb.shape

(torch.Size([784]), torch.Size([1]))

### Validation

In [6]:
valid_x = torch.cat([
    torch.stack([
        tensor(Image.open(o)).float()/255
        for o in (path/'valid'/'3').ls()
    ]),
    torch.stack([
        tensor(Image.open(o)).float()/255
        for o in (path/'valid'/'7').ls()
    ])
]).view(-1, 28*28)

valid_y = tensor([1]*len((path/'valid'/'3').ls()) + [0]*len((path/'valid'/'7').ls())).unsqueeze(1)

valid_dset = list(zip(valid_x, valid_y))
x,y = valid_dset[0]
x.shape, y

(torch.Size([784]), tensor([1]))

In [7]:
valid_dl = DataLoader(valid_dset)

xb,yb = first(valid_dl)
xb.shape, yb.shape

(torch.Size([784]), torch.Size([1]))

## Functions

In [9]:
def mnist_loss(preds, targets):
    preds = preds.sigmoid()
    return torch.where(targets==1, 1-preds, preds).mean()

mnist_loss(torch.tensor([0.9, 0.4, 0.2]), tensor([1,0,1]))

tensor(0.4460)

In [10]:
def mnist_accuracy(xb, yb):
    preds = xb.sigmoid()
    correct = (preds>0.5) == yb
    return correct.float().mean()

mnist_accuracy(torch.tensor([0.9, 0.4, 0.2]), tensor([1,0,1]))

tensor(0.6667)

## Putting it all togather

In [11]:
dls = DataLoaders(dl, valid_dl)

In [12]:
dls.train.one_batch()[0].shape

torch.Size([784])

In [13]:
linear_model = nn.Linear(28*28, 1)

learner = MyLearner(dls, linear_model, mnist_loss, mnist_accuracy)

learner.fit(20, 0.001)

Epoch Train_Loss Valid_Accuracy  Time
Train loops: 12396
 00:    0.0023     0.5118   3.5886
Train loops: 12396
 01:    0.0007     0.8292   3.7498
Train loops: 12396
 02:    0.0004     0.9190   4.5181
Train loops: 12396
 03:    0.0003     0.9426   4.7744
Train loops: 12396
 04:    0.0002     0.9563   3.5970
Train loops: 12396
 05:    0.0002     0.9661   3.4616
Train loops: 12396
 06:    0.0002     0.9676   3.7780
Train loops: 12396
 07:    0.0001     0.9676   3.9274
Train loops: 12396
 08:    0.0001     0.9701   3.5247
Train loops: 12396
 09:    0.0001     0.9711   3.9182
Train loops: 12396
 10:    0.0001     0.9725   3.5024
Train loops: 12396
 11:    0.0001     0.9730   3.6048
Train loops: 12396
 12:    0.0001     0.9745   3.4185
Train loops: 12396
 13:    0.0001     0.9750   3.3832
Train loops: 12396
 14:    0.0001     0.9755   3.4099
Train loops: 12396
 15:    0.0001     0.9755   3.3242
Train loops: 12396
 16:    0.0001     0.9769   3.3767
Train loops: 12396
 17:    0.0001     0.9769

In [14]:
nn_basic = nn.Sequential(
    nn.Linear(28*28, 30),
    nn.ReLU(),
    nn.Linear(30, 1),
)

learner = MyLearner(dls, nn_basic, mnist_loss, mnist_accuracy)

learner.fit(20, 0.001)

Epoch Train_Loss Valid_Accuracy  Time
Train loops: 12396
 00:    0.0037     0.5044   5.4948
Train loops: 12396
 01:    0.0006     0.7512   5.5774
Train loops: 12396
 02:    0.0002     0.8813   5.5449
Train loops: 12396
 03:    0.0001     0.9181   5.4589
Train loops: 12396
 04:    0.0001     0.9362   5.5031
Train loops: 12396
 05:    0.0000     0.9500   5.4912
Train loops: 12396
 06:    0.0000     0.9578   5.4871
Train loops: 12396
 07:    0.0000     0.9607   5.5752
Train loops: 12396
 08:    0.0000     0.9652   5.6467
Train loops: 12396
 09:    0.0000     0.9681   5.5447
Train loops: 12396
 10:    0.0000     0.9691   5.4525
Train loops: 12396
 11:    0.0000     0.9706   5.2736
Train loops: 12396
 12:    0.0000     0.9735   5.1758
Train loops: 12396
 13:    0.0000     0.9745   5.3209
Train loops: 12396
 14:    0.0000     0.9750   5.5401
Train loops: 12396
 15:    0.0000     0.9755   5.3954
Train loops: 12396
 16:    0.0000     0.9760   5.3875
Train loops: 12396
 17:    0.0000     0.9764

# Complete MNIST

## Getting data

In [24]:
path = untar_data(URLs.MNIST)
path.ls()

(#2) [Path('/home/james/.fastai/data/mnist_png/testing'),Path('/home/james/.fastai/data/mnist_png/training')]

In [25]:
train_path = (path/'training').ls().sorted()

all_data = [(tensor(Image.open(o)), int(path.name))
            for path in train_path
            for o in (path).ls()]

all_data

# shuffle

random.shuffle(all_data)


split = int(len(all_data)*0.8)

train_data = all_data[:split]
valid_data = all_data[split:]

len(train_data), len(valid_data)

(48000, 12000)

In [26]:
def get_dl(data_list):
    x = torch.stack([o[0] for o in data_list]).float()/255
    x = x.view(-1, 28*28)
    y = tensor([o[1] for o in data_list]).unsqueeze(1)
    print(x.shape, y.shape)

    dset = list(zip(x,y))

    print(len(dset))

    dl = DataLoader(dset, batch_size=64)

    return dl

train_dl = get_dl(train_data)
valid_dl = get_dl(valid_data)

train_dl.one_batch()[0].shape, valid_dl.one_batch()[1].shape

torch.Size([48000, 784]) torch.Size([48000, 1])
48000
torch.Size([12000, 784]) torch.Size([12000, 1])
12000


(torch.Size([64, 784]), torch.Size([64, 1]))

In [27]:
def visualize_samples(batch, num_samples=10):
    # Get a batch of data from the data loader
    images, labels = batch
    images = images.view(-1, 28, 28)  # Reshape images to 28x28
    
    # Plot the specified number of samples
    fig, axes = plt.subplots(1, num_samples, figsize=(15, 3))
    for i in range(num_samples):
        axes[i].imshow(images[i].numpy(), cmap='gray')
        axes[i].set_title(f'Label: {labels[i].item()}')
        axes[i].axis('off')
    plt.show()

# Visualize samples from training and validation data loaders
visualize_samples(train_dl.one_batch())
visualize_samples(valid_dl.one_batch())


<Figure size 1500x300 with 10 Axes>

<Figure size 1500x300 with 10 Axes>

In [28]:
dls = DataLoaders(train_dl, valid_dl)

## Needed function

In [29]:
mnist_model = nn.Sequential(
    nn.Linear(28*28, 30),
    nn.ReLU(),
    nn.Linear(30, 10)
)

# mnist_model = nn.Sequential(
#     nn.Linear(28*28, 128),
#     nn.ReLU(),
#     nn.Linear(128, 64),
#     nn.ReLU(),
#     nn.Linear(64, 10)
# )

mnist_model(train_dl.one_batch()[0]).shape

torch.Size([64, 10])

In [30]:
preds = mnist_model(train_dl.one_batch()[0])
targets = train_dl.one_batch()[1]

preds.shape, targets.shape

(torch.Size([64, 10]), torch.Size([64, 1]))

In [31]:
targets.view(-1)

tensor([2, 1, 3, 5, 1, 1, 2, 6, 8, 0, 4, 2, 7, 7, 2, 1, 1, 9, 0, 4, 8, 8, 7, 0,
        3, 6, 0, 1, 7, 3, 5, 0, 6, 8, 9, 1, 9, 9, 7, 9, 1, 2, 8, 8, 8, 6, 3, 9,
        5, 4, 9, 2, 6, 3, 1, 8, 4, 2, 5, 5, 5, 3, 6, 3])

In [32]:
def complete_mnist_loss(preds, targets):
    # preds = preds.softmax(1)
    # display(preds)
    # targets = targets.view(-1)
    # targets_probs = preds[range(targets.shape[0]), targets]

    # display(targets_probs)
    # return -torch.log(targets_probs).mean()

    # print(preds.shape, targets.shape)
    # print(targets * preds.log())
    # return -(targets * preds.log()).sum() / len(preds)
    return nn.functional.cross_entropy(preds, targets.squeeze())
    
random_preds = tensor([[0.9261, 0.7523, 0.7815, 0.5587, 0.9547, 0.3564, 0.9957, 0.5457, 0.9605,
         0.7544],
        [0.7494, 0.8851, 0.9722, 0.9944, 0.3382, 0.7579, 0.5753, 0.2673, 0.1166,
         0.3712],
        [0.0948, 0.6444, 0.0074, 0.1551, 0.0752, 0.8233, 0.2820, 0.0842, 0.5581,
         0.2289],
        [0.2603, 0.6603, 0.6184, 0.3055, 0.7156, 0.6018, 0.3889, 0.0617, 0.7188,
         0.6093],
        [0.6258, 0.9553, 0.1366, 0.4280, 0.8662, 0.4507, 0.7663, 0.3752, 0.7364,
         0.8468]])
display(random_preds)
random_targets = tensor([9, 5, 7, 4, 1])
display(random_targets)
complete_mnist_loss(random_preds, random_targets)

tensor([[0.9261, 0.7523, 0.7815, 0.5587, 0.9547, 0.3564, 0.9957, 0.5457, 0.9605,
         0.7544],
        [0.7494, 0.8851, 0.9722, 0.9944, 0.3382, 0.7579, 0.5753, 0.2673, 0.1166,
         0.3712],
        [0.0948, 0.6444, 0.0074, 0.1551, 0.0752, 0.8233, 0.2820, 0.0842, 0.5581,
         0.2289],
        [0.2603, 0.6603, 0.6184, 0.3055, 0.7156, 0.6018, 0.3889, 0.0617, 0.7188,
         0.6093],
        [0.6258, 0.9553, 0.1366, 0.4280, 0.8662, 0.4507, 0.7663, 0.3752, 0.7364,
         0.8468]])

tensor([9, 5, 7, 4, 1])

tensor(2.2334)

In [33]:
def complete_mnist_accuracy(preds, targets):
    preds = preds.argmax(dim=1)
    targets = targets.squeeze()
    correct = (preds == targets).float()
    return correct.float().mean()

random_preds = torch.rand((5, 10))
display(random_preds)
random_targets = torch.randint(0, 10, (5, 1))
display(random_targets)
complete_mnist_accuracy(random_preds, random_targets)

tensor([[0.2472, 0.8071, 0.6438, 0.7001, 0.5046, 0.4333, 0.2999, 0.5597, 0.7503,
         0.1459],
        [0.8508, 0.1171, 0.0273, 0.2872, 0.3250, 0.4297, 0.2196, 0.7902, 0.1327,
         0.8547],
        [0.6367, 0.8120, 0.8256, 0.7937, 0.3605, 0.4780, 0.1094, 0.8865, 0.8531,
         0.1587],
        [0.8184, 0.6203, 0.0273, 0.8205, 0.5324, 0.4460, 0.1424, 0.0323, 0.9509,
         0.7708],
        [0.0875, 0.9155, 0.5544, 0.0337, 0.9919, 0.0979, 0.7772, 0.7446, 0.5790,
         0.3651]])

tensor([[4],
        [0],
        [0],
        [2],
        [7]])

tensor(0.)

In [46]:

learner = MyLearner(dls, mnist_model, complete_mnist_loss, complete_mnist_accuracy)

learner.fit(20, 0.01)

Epoch Train_Loss Valid_Accuracy  Time
Train loops: 750
 00:    0.2236     0.9364   0.9260
Train loops: 750
 01:    0.2204     0.9377   0.7898
Train loops: 750
 02:    0.2169     0.9386   0.9310
Train loops: 750
 03:    0.2137     0.9399   0.6557
Train loops: 750
 04:    0.2108     0.9405   0.5789
Train loops: 750
 05:    0.2075     0.9408   0.5567
Train loops: 750
 06:    0.2043     0.9415   0.5665
Train loops: 750
 07:    0.2016     0.9423   0.5567
Train loops: 750
 08:    0.1983     0.9427   0.5778
Train loops: 750
 09:    0.1954     0.9432   0.6502
Train loops: 750
 10:    0.1927     0.9439   0.5802
Train loops: 750
 11:    0.1896     0.9446   0.5839
Train loops: 750
 12:    0.1865     0.9451   0.5784
Train loops: 750
 13:    0.1837     0.9457   0.7784
Train loops: 750
 14:    0.1806     0.9466   0.6121
Train loops: 750
 15:    0.1779     0.9470   0.6739
Train loops: 750
 16:    0.1748     0.9473   0.6932
Train loops: 750
 17:    0.1719     0.9481   0.6989
Train loops: 750
 18:    0

In [47]:
learner.predict(train_dl.one_batch()[0][0])

2

In [ ]:
train_dl.one_batch()[1].squeeze(
)

tensor([9, 8, 9, 6, 3, 1, 3, 5, 9, 5, 1, 4, 3, 3, 6, 6, 6, 8, 9, 1, 7, 1, 5, 8,
        0, 5, 3, 5, 9, 9, 8, 2, 9, 8, 7, 0, 8, 0, 4, 4, 2, 1, 1, 7, 6, 2, 6, 8,
        5, 6, 2, 9, 6, 9, 7, 8, 0, 4, 1, 0, 7, 7, 6, 0], device='cuda:0')

In [ ]:
mnist_model(train_dl.one_batch()[0]).argmax(dim=1)

tensor([9, 3, 9, 0, 3, 1, 3, 3, 9, 3, 1, 0, 3, 3, 3, 0, 6, 3, 9, 1, 7, 1, 3, 3,
        0, 3, 3, 2, 9, 9, 3, 0, 3, 3, 7, 0, 3, 3, 9, 0, 2, 1, 3, 7, 6, 2, 6, 3,
        3, 3, 3, 9, 3, 9, 3, 7, 0, 9, 3, 0, 7, 7, 0, 0], device='cuda:0')

# Gradio app

In [61]:
import gradio as gr
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

def classify_image(img):
    # Convert the image to a PIL image if it's not already
    if not isinstance(img, Image.Image):
        img = Image.fromarray(img)

    # Define the transformations
    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),  # Ensure the image is grayscale
        transforms.Resize((28, 28)),  # Resize to 28x28
        transforms.ToTensor(),  # Convert to tensor
        transforms.Normalize((0.5,), (0.5,))  # Normalize the image
    ])
    
    # Apply the transformations
    img_tensor = transform(img)  

    # De-normalize the image for proper visualization
    img_tensor = img_tensor * 0.5 + 0.5

    # Convert the tensor to a numpy array and then to a PIL image
    np_img = img_tensor.numpy().squeeze() * 255
    transformed_img = Image.fromarray(np_img.astype(np.uint8))

    # Flatten the tensor for prediction
    img_tensor = img_tensor.view(-1, 28 * 28)
    print(img_tensor)

    prediction = learner.predict(img_tensor)[0]
    
    return prediction, transformed_img

app = gr.Interface(fn=classify_image, inputs="image", outputs=["label", "image"])

app.launch()

Running on local URL:  http://127.0.0.1:7885

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 3.50.2, however version 4.29.0 is available, please upgrade.
--------
tensor([[0.6745, 0.6706, 0.6667, 0.6667, 0.6706, 0.6667, 0.6667, 0.6627, 0.6588,
         0.6588, 0.6627, 0.6627, 0.6627, 0.6588, 0.6510, 0.6549, 0.6588, 0.6588,
         0.6549, 0.6549, 0.6549, 0.6510, 0.6510, 0.6510, 0.6510, 0.6549, 0.6549,
         0.6510, 0.6784, 0.6706, 0.6627, 0.6667, 0.6667, 0.6667, 0.6706, 0.6667,
         0.6627, 0.6588, 0.6588, 0.6627, 0.6588, 0.5843, 0.5294, 0.6431, 0.6549,
         0.6549, 0.6510, 0.6549, 0.6549, 0.6471, 0.6510, 0.6510, 0.6549, 0.6549,
         0.6549, 0.6471, 0.6745, 0.6706, 0.6667, 0.6667, 0.6667, 0.6667, 0.6706,
         0.6706, 0.6627, 0.6588, 0.6588, 0.6588, 0.6157, 0.4392, 0.5647, 0.6471,
         0.6549, 0.6549, 0.6549, 0.6549, 0.6510, 0.6471, 0.6471, 0.6471, 0.6510,
         0.6549, 0.6549, 0.6510, 0.6784, 0.6745, 0.6667, 0.6667, 0.6667, 0.6667,
         0.6706, 0.6706, 0.6667, 0.6588, 0.6627, 0.6549, 0.4627, 0.5176, 0.6510,